In [ ]:
#IMPORT FUNCTIONS
from m6Predict_WhistleFunctions import *
import matplotlib.pyplot as plt
plt.switch_backend('agg')

In [ ]:
#set variables
modelFlank=80
numOut_Classes=1
GPU=2
modelFn='model_weights_BEST.h5'
#add GPU 

In [ ]:
#test
halfFlank=int(modelFlank/2) 

InpF=h5py.File('Inp_test.h5','r')
TgtF=h5py.File('Tgt_test.h5','r')

X=[]
Y=[]
for k in InpF.keys():
    X.append(InpF[k][:,5000-halfFlank:-5000+halfFlank])
    Y.append(TgtF[k][:])
tstInpBlocks=np.concatenate(X,axis=0)
tstTgtBlocks=np.concatenate(Y,axis=0)

rem = tstInpBlocks.shape[0] % GPU
if rem != 0:
    tstInpBlocks=tstInpBlocks[:-rem]
    tstTgtBlocks=tstTgtBlocks[:-rem]

model=load_model(modelFn,custom_objects={'tf':tf})
#calculate top-k and PR-AUC accuracies for SDs and SAs (seperatly)

p=model.predict(tstInpBlocks)
print p.shape,tstInpBlocks.shape,tstTgtBlocks.shape

Tks=[]
PRs=[]
if numOut_Classes > 1:
    cRange=range(1,numOut_Classes)
else:
    cRange=[0]
for c in cRange:
    Fullc=tstTgtBlocks[:,:,c].flatten()
    S=[i for i in range(len(Fullc)) if Fullc[i] == 1]
    pS=np.argsort(p[:,:,c].flatten())[-len(S):]
    pS=pd.Series(pS)
    nCrct=pS[pS.isin(S)].size
    Tk=nCrct/float(len(S))
    Tks.append(Tk)
    
    PR=average_precision_score(Fullc,p[:,:,c].flatten())
    PRs.append(PR)
    if numOut_Classes < 3:
        Tks.append(0.0)
        PRs.append(0.0)
writeAccs=str(Tks[0])+','+str(Tks[1])+','+str(PRs[0])+','+str(PRs[1])
open('Tks_PRs.txt', 'w').write(writeAccs)

In [ ]:
# graph Precision Recall of m6A sites
precision, recall, _ = precision_recall_curve(tstTgtBlocks[:,:,0].flatten(),p[:,:,0].flatten())


plt.step(recall, precision, color='b', alpha=0.2,
         where='post')
plt.fill_between(recall, precision, alpha=0.2, color='b')
plt.plot([0,1],[PRs[0],PRs[0]])

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.title('m6A Sites')

plt.savefig('m6A_Sites.png')